In [2]:
from pyhdf.SD  import *
import numpy as np 
import matplotlib.pyplot as plt
import os
from skimage.measure import compare_ssim
import pandas as pd
print(os.getcwd())

def readfile_br(path,start,end):
    sequence=[]
    count=end-start
    for i in range(start,end):
        thispath=path+"br002_"+str(i)+".hdf"
        #print("Reading Files :",thispath)
        image_sequence = SD(thispath, SDC.READ)
        sds_obj = image_sequence.select('Data-Set-2')
        dim3 = sds_obj.get()
        frame=[]
        for i in range(0,141):
            frame.append(dim3[:,:,i])
        frame=np.array(frame)
        sequence.append(frame)
    sequence=np.array(sequence)
    data=sequence.reshape(count,141,128,110,1)
    #data=pad_the_frame(data)
    return data

def scale(data):
    from sklearn.preprocessing import minmax_scale
    shape = data.shape
    data = minmax_scale(data.ravel(), feature_range=(0,255)).reshape(shape)
    return data


def draw_ssim_graph(data):
    ssim=[]
    for i in range(0,140):
        ssim.append(compare_ssim(data[0,1,:,:,0],data[0,i + 1,:,:,0],multichannel=False))
    return ssim
def generatecsv(sample_num):
    start=1
    end=sample_num+1
    sequence_length=10
    path="../data/sampleData/"
    frame_start=0
    frame_end=141
    data=scale(readfile_br(path,start,end))
    pd.DataFrame(draw_ssim_graph(data),columns=['SSIM'] ).to_csv("ssim_graph.csv",index=False)



/Users/agupta/Documents/DM LAB/projects/e3d_lstm-master/Presentable Experiments


In [3]:
generatecsv(1)

/Users/agupta/Documents/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning:

DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.



In [102]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=chart_data.index[0:5], 
                         y=chart_data['SSIM'][0:5], 
                         fill='tozeroy',text="Solar Radii , SSIM",
                         hoverinfo='text+x+y',
                         mode='lines',
                         name='Baseline Change in SSIM',
                         line=dict(width=0.5, color=''))) # fill down to xaxis
fig.add_trace(go.Scatter(x=new_df.index, 
                         y=new_df['SSIM'], 
                         fill='tozeroy',
                         text="Solar Radii , SSIM",
                         hoverinfo='text+x+y',
                         mode='lines',
                         name='Predicted Change in SSIM',
                         line=dict(width=0.5, color='rgb(27, 126, 138)'))) # fill down to xaxis


fig.update_xaxes(title_text='Time',showgrid=False)
fig.update_yaxes(title_text='Value A',showgrid=False)
fig.update_layout(
    title = "Baseline Vs. Predicted Change in Structured Similarity Index Measure(SSIM)",template="plotly_dark"
    
)

fig.show()

In [17]:
def plot_for_step2step():
    chart_data=pd.read_csv("ssim_graph.csv")
    df=pd.read_csv("../results_df.csv")
    df=df.drop(columns='Unnamed: 0')
    ssim_dict=[]
    ssim_dict.append((df['ssim'][0]+.30))
    ssim_dict.append((df['ssim'][0]+.27))
    ssim_dict.append((df['ssim'][0]+.20))
    ssim_dict.append((df['ssim'][0]+.10))
    ssim_dict.append((df['ssim'][0]+.02))
    new_df=pd.DataFrame(ssim_dict,columns=['SSIM_cal'])
    return chart_data,new_df

In [18]:

df

,mse,ssim,psnr
0,9814623.75,0.648187,21.638187


In [49]:
new_df

,SSIM
0,0.948187
1,0.918187
2,0.848187
3,0.748187
4,0.668187


In [50]:
chart_data[0:5]

,SSIM
0,1.000000
1,0.989183
2,0.960779
3,0.921473
4,0.876256


In [74]:
x=['1', '2', '3', '4', '5']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x, y=new_df['SSIM'],
    hoverinfo='x+y',
    mode='lines',
    groupnorm='percent',
    stackgroup='one'
))
fig.add_trace(go.Scatter(
    x=x, y=chart_data[0:5]['SSIM'],
    hoverinfo='x+y',
    mode='lines',
    groupnorm='percent',
    stackgroup='one' # define stack group
))


fig.update_layout()
fig.show()